In [8]:
40 & 0x0fff

40

In [11]:
bin(0x0fff)

'0b111111111111'

In [16]:
print(bin(4097), '&', bin(0x0fff), '=', bin(4097&0x0fff))

0b1000000000001 & 0b111111111111 = 0b1


In [85]:
number = 5620042320032005
print(bin(number))
number &= 0xf000
bin(number >> 12)

0b10011111101110110011000010010000110111011000100000101


'0b1011'

In [80]:
len(bin(5620042320032005))

55

In [82]:
hex(0b1111111111111111100000000000000000000)

'0x1ffff00000'

In [67]:
hex(0b1111000000000000)

'0xf000'

In [9]:
def encoder(filein,fileout):
    blocksize=8
    while True:
        s=filein.read(blocksize)
        if not s:
            break
        fileout.write(s)

In [10]:
def decoder(filein, fileout, filelog):
    blocksize = 8
    s = filein.read(blocksize)
    if not s:
        return
    while True:
        fileout.write(s)
        s = filein.read(blocksize)
        if not s:
            break

In [90]:
type("State".encode())

bytes

In [91]:
pos = 0
state = 0

In [92]:
print(f"Pos: {pos}" + f" State: {state}" + 
            f"time")

Pos: 0State: 0time
